In [ ]:
# default_exp run_params

In [ ]:
#export
import pytorch_lightning as pl

# RunParams

> An object that contains all the parameters necessary to run an `Experiment`. 

In [ ]:
#export
class Model(pl.LightningModule):
    def __init__(self):
        pass

    def prepare_data(self):
        pass

    def forward(self, x):
        pass

    def training_step(self, batch, batch_idx):
        pass

    def validation_step(self, batch, batch_idx):
        pass

    def validation_epoch_end(self, outputs):
        pass

    def train_dataloader(self):
        pass

    def val_dataloader(self):
        pass

    def configure_optimizer(self):
        pass

# Testing

In [ ]:
# Creation
model = Model()
assert isinstance(model, Model)